In [1]:
{-# LANGUAGE InstanceSigs #-}

newtype Parser a = Parser { runParser :: String -> [(a, String)] }

In [2]:
instance Functor Parser where
    fmap :: (a -> b) -> Parser a -> Parser b
    fmap f parser = Parser $ \s -> let
        parsed    = runParser parser s
        fn (a, s) = (f a, s)
        in map fn parsed

In [3]:
import Control.Applicative (Alternative(empty, (<|>)))

instance Applicative Parser where
    pure :: a -> Parser a
    pure a = Parser (\s -> [(a, s)])
    (<*>) :: Parser (a -> b) -> Parser a -> Parser b
    f <*> a = Parser $ \s ->
        [ (f' a', s'')
        | (f', s')  <- runParser f s
        , (a', s'') <- runParser a s'
        ]

instance Alternative Parser where
    empty :: Parser a
    empty = Parser $ \_ -> []
    
    (<|>) :: Parser a -> Parser a -> Parser a
    a <|> b = Parser $ \s -> (runParser a s) ++ (runParser b s)

In [4]:
instance Monad Parser where
    (>>=) :: Parser a -> (a -> Parser b) -> Parser b
    a >>= f = Parser $ \s -> concat
        [ runParser (f a') s'
        | (a', s') <- runParser a s
        ]

In [16]:
import Control.Monad (guard)

anyChar :: Parser Char
anyChar = Parser $ \s -> case s of
    []     -> []
    (c:cs) -> [(c, cs)]

satisfy :: (Char -> Bool) -> Parser Char
satisfy pred = do
    c <- anyChar
    guard $ pred c
    return c

char :: Char -> Parser Char
char c = satisfy (c==)

string :: String -> Parser String
string []     = pure []
string (c:cs) = (:) <$> char c <*> string cs

In [12]:
(+++)  :: Parser a -> Parser a -> Parser a
p +++ q = Parser $ \s -> case runParser (p <|> q) s of
    []     -> []
    (x:xs) -> [x]

many :: Parser a -> Parser [a]
many p = many1 p +++ pure []

many1 :: Parser a -> Parser [a]
many1 p = (:) <$> p <*> many p

In [15]:
runParser (many (char 'b')) "a"

[("","a")]

In [18]:
runParser (string "foo" *> string "bar") "foobar"

[("bar","")]